In [21]:
import pandas as pd
import pypsa
import logging
import numpy as np

In [19]:
def apply_time_segmentation(n, segments, solver_name="cbc"):
    print(f"Aggregating time series to {segments} segments.")
    try:
        import tsam.timeseriesaggregation as tsam
    except:
        raise ModuleNotFoundError("Optional dependency 'tsam' not found."
                                "Install via 'pip install tsam'")

    p_max_pu_norm = n.generators_t.p_max_pu.max()
    p_max_pu = n.generators_t.p_max_pu / p_max_pu_norm

    load_norm = n.loads_t.p_set.max()
    load = n.loads_t.p_set / load_norm
    
    inflow_norm = n.storage_units_t.inflow.max()
    inflow = n.storage_units_t.inflow / inflow_norm

    raw = pd.concat([p_max_pu, load, inflow], axis=1, sort=False)

    agg = tsam.TimeSeriesAggregation(raw, hoursPerPeriod=len(raw),
                                    noTypicalPeriods=1, noSegments=int(segments),
                                    segmentation=True, solver=solver_name)

    segmented = agg.createTypicalPeriods()

    weightings = segmented.index.get_level_values("Segment Duration")
    offsets = np.insert(np.cumsum(weightings[:-1]), 0, 0)
    snapshots = [n.snapshots[0] + pd.Timedelta(f"{offset}h") for offset in offsets]

    n.set_snapshots(pd.DatetimeIndex(snapshots, name='name'))
    n.snapshot_weightings = pd.Series(weightings, index=snapshots, name="weightings", dtype="float64")
    
    segmented.index = snapshots
    n.generators_t.p_max_pu = segmented[n.generators_t.p_max_pu.columns] * p_max_pu_norm
    n.loads_t.p_set = segmented[n.loads_t.p_set.columns] * load_norm
    n.storage_units_t.inflow = segmented[n.storage_units_t.inflow.columns] * inflow_norm

    return n

In [34]:
n= pypsa.Network('../networks/elec_csir-aggressive_redz_LC_27-supply.nc')

INFO:pypsa.io:Imported network elec_csir-aggressive_redz_LC_27-supply.nc has buses, carriers, generators, lines, loads, storage_units


In [ ]:
buses_i = buses.get(carrier, n.buses.index)

In [22]:
n = apply_time_segmentation(n, 200, solver_name='cbc')

Aggregating time series to 200 segments.


INFO:pypsa.components:Applying weightings to all columns of `snapshot_weightings`


In [36]:
n.generators

,Owner,carrier,p_nom,y,x,efficiency,marginal_cost,capital_cost,ramp_limit_up,bus,...,start_up_cost,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
Kendal,Eskom,coal,3840.0,-26.08805,28.96888,0.306305,275.605800,5.940000e+05,0.028125,HIGHVELD NORTH,...,0.0,0.0,0,0,1,0,NaN,1.0,1.0,0.0
Lethabo,Eskom,coal,3540.0,-26.74027,27.97500,0.328018,179.920000,5.940000e+05,0.100000,VAAL TRIANGLE,...,0.0,0.0,0,0,1,0,NaN,1.0,1.0,0.0
MajubaDry,Eskom,coal,1830.0,-27.09555,29.77055,0.306305,313.215400,5.940000e+05,0.055738,NEWCASTLE,...,0.0,0.0,0,0,1,0,NaN,1.0,1.0,0.0
MajubaWet,Eskom,coal,2010.0,-27.09555,29.77055,0.327154,313.393200,5.940000e+05,0.056716,NEWCASTLE,...,0.0,0.0,0,0,1,0,NaN,1.0,1.0,0.0
Matimba,Eskom,coal,3720.0,-23.66777,27.61277,0.308907,189.855600,5.940000e+05,0.048387,WATERBERG,...,0.0,0.0,0,0,1,0,NaN,1.0,1.0,0.0
Medupi,Eskom,coal,722.0,-23.42000,27.33000,0.366898,239.935600,4.700000e+06,0.598338,WATERBERG,...,0.0,0.0,0,0,1,0,NaN,1.0,1.0,0.0
Tutuka,Eskom,coal,3480.0,-26.77565,29.35212,0.329731,340.868000,5.940000e+05,0.055172,HIGHVELD SOUTH,...,0.0,0.0,0,0,1,0,NaN,1.0,1.0,0.0
Koeberg,Eskom,nuclear,1860.0,-33.67366,18.42811,0.324003,126.999100,9.680000e+05,NaN,PENINSULA,...,0.0,0.0,0,0,1,0,NaN,1.0,1.0,0.0
Municipal_Coal,South African Municipality,coal,160.0,-26.65800,28.11380,0.290980,249.496400,0.000000e+00,0.187500,NIGEL,...,0.0,0.0,0,0,1,0,NaN,1.0,1.0,0.0


In [37]:
n.storage_units

,Owner,carrier,p_nom,y,x,capital_cost,bus,efficiency_store,efficiency_dispatch,max_hours,...,marginal_cost,build_year,lifetime,state_of_charge_initial,state_of_charge_initial_per_period,state_of_charge_set,cyclic_state_of_charge_per_period,standing_loss,inflow,p_nom_opt
StorageUnit,,,,,,,,,,,,,,,,,,,,,
Drakensberg,Eskom,PHS,1000.0,-28.56283,29.08275,2.010000e+05,LADYSMITH,0.858487,0.858487,21.700000,...,0.0,0,inf,0.0,False,NaN,True,0.0,0.0,0.0
Ingula,Eskom,PHS,1332.0,-28.16500,29.35120,2.530000e+06,NEWCASTLE,0.883176,0.883176,20.570571,...,0.0,0,inf,0.0,False,NaN,True,0.0,0.0,0.0
Palmiet,Eskom,PHS,400.0,-34.19722,18.97361,2.010000e+05,PENINSULA,0.882610,0.882610,25.000000,...,0.0,0,inf,0.0,False,NaN,True,0.0,0.0,0.0
Gariep,Eskom,hydro,360.0,-30.62396,25.50403,0.000000e+00,KAROO,1.000000,1.000000,20.567643,...,0.0,0,inf,0.0,False,NaN,True,0.0,0.0,0.0
Vanderkloof,Eskom,hydro,240.0,-29.99337,24.73384,0.000000e+00,KAROO,1.000000,1.000000,20.567643,...,0.0,0,inf,0.0,False,NaN,True,0.0,0.0,0.0
ColleyWobbles,Eskom,hydro,65.0,-32.05000,28.58333,0.000000e+00,EAST LONDON,1.000000,1.000000,20.567643,...,0.0,0,inf,0.0,False,NaN,True,0.0,0.0,0.0
Steenbras,South African Municipality,PHS,180.0,-34.15300,18.90000,0.000000e+00,PENINSULA,0.848528,0.848528,15.000000,...,0.0,0,inf,0.0,False,NaN,True,0.0,0.0,0.0
CahoraBassa,Import from Mozambique,hydro,1500.0,NaN,NaN,0.000000e+00,POLOKWANE,1.000000,1.000000,20.567643,...,0.0,0,inf,0.0,False,NaN,True,0.0,0.0,0.0
NAMAQUALAND battery,,battery,0.0,NaN,NaN,1.083978e+05,NAMAQUALAND,NaN,NaN,4.000000,...,0.0,0,inf,0.0,False,NaN,True,0.0,0.0,0.0


In [3]:
n= pypsa.Network('../results/version-0.5/networks/csir-aggressive_redz_E_LC_27-supply.nc')

INFO:pypsa.io:Imported network csir-aggressive_redz_E_LC_27-supply.nc has buses, carriers, generators, lines, loads, storage_units


In [6]:
n.generators.p_nom_opt.groupby(n.generators.carrier).sum()

carrier
CCGT       8.306192e+03
Coal       1.990200e+04
Nuclear    1.860000e+03
OCGT      -9.707740e-13
PV         8.767739e+04
Wind       4.248762e+04
load       2.700000e+07
Name: p_nom_opt, dtype: float64

In [10]:
n.generators_t.p['PRETORIA Load'].max()

0.0

In [41]:
def apply_time_segmentation(n, segments, solver_name="cbc"):
    logger.info(f"Aggregating time series to {segments} segments.")
    try:
        import tsam.timeseriesaggregation as tsam
    except:
        raise ModuleNotFoundError("Optional dependency 'tsam' not found."
                                "Install via 'pip install tsam'")

    p_max_pu_norm = n.generators_t.p_max_pu.max()
    p_max_pu = n.generators_t.p_max_pu / p_max_pu_norm

    load_norm = n.loads_t.p_set.max()
    load = n.loads_t.p_set / load_norm
    
    inflow_norm = n.storage_units_t.inflow.max()
    inflow = n.storage_units_t.inflow / inflow_norm

    raw = pd.concat([p_max_pu, load, inflow], axis=1, sort=False)

    agg = tsam.TimeSeriesAggregation(raw, hoursPerPeriod=len(raw),
                                    noTypicalPeriods=1, noSegments=int(segments),
                                    segmentation=True, solver=solver_name)

    segmented = agg.createTypicalPeriods()

    weightings = segmented.index.get_level_values("Segment Duration")
    offsets = np.insert(np.cumsum(weightings[:-1]), 0, 0)
    snapshots = [n.snapshots[0] + pd.Timedelta(f"{offset}h") for offset in offsets]

    n.set_snapshots(pd.DatetimeIndex(snapshots, name='name'))
    n.snapshot_weightings = pd.Series(weightings, index=snapshots, name="weightings", dtype="float64")
    
    segmented.index = snapshots
    n.generators_t.p_max_pu = segmented[n.generators_t.p_max_pu.columns] * p_max_pu_norm
    n.loads_t.p_set = segmented[n.loads_t.p_set.columns] * load_norm
    n.storage_units_t.inflow = segmented[n.storage_units_t.inflow.columns] * inflow_norm

    return n